## LinkML and SQL Alchemy Tutorial

This tutorial walks through the use of LinkML in combination with SQLite3 and SQL Alchemy.

The steps are:

 - Create a SQL schema from a LinkML schema (the standard PersonInfo example)
 - Generate SQL Alchemy "Models" from the same LinkML schema
 - Create instances of these Models and insert into database
 - Query using direct SQL
 - Query using SQL Alchemy ORM

## PersonInfo schema

We use the standard PersonInfo schema bundled in the LinkML repo:

In [1]:
!cat ../examples/PersonSchema/personinfo.yaml

id: https://w3id.org/linkml/examples/personinfo
name: personinfo
description: |-
  Information about people, based on [schema.org](http://schema.org)
license: https://creativecommons.org/publicdomain/zero/1.0/
default_curi_maps:
  - semweb_context
imports:
  - linkml:types
prefixes:
  personinfo: https://w3id.org/linkml/examples/personinfo/
  linkml: https://w3id.org/linkml/
  schema: http://schema.org/
  rdfs: http://www.w3.org/2000/01/rdf-schema#
  prov: http://www.w3.org/ns/prov#
  GSSO: http://purl.obolibrary.org/obo/GSSO_
  famrel: https://example.org/FamilialRelations#
  # DATA PREFIXES
  P: http://example.org/P/
  ROR: http://example.org/ror/
  CODE: http://example.org/code/
  GEO: http://example.org/geoloc/
default_prefix: personinfo
default_range: string

emit_prefixes:
  - rdf
  - rdfs
  - xsd
  - skos

classes:

  NamedThing:
    description: >-
      A generic grouping for any identifiable entity
    slots:
      - id
      - name
      - description
      - image
    close

## Create a working directory

For demonstration purposes we will put all generated artefacts in a folder `tmp`.

Note that if you were doing this in a real project, you should use the linkml project cookiecutter
to create a standard project structure

In [2]:
!mkdir -p tmp

## Generate SQL CREATE TABLE statements

We use the [gen-sqlddl](https://linkml.io/linkml/generators/sqltable.html) command to create a SQL DDL (CREATE TABLE statements):

In [3]:
!gen-sqlddl ../examples/PersonSchema/personinfo.yaml > tmp/personinfo.ddl.sql

In [4]:
!grep -A20 'CREATE TABLE "Person"' tmp/personinfo.ddl.sql

CREATE TABLE "Person" (
	primary_email TEXT, 
	birth_date TEXT, 
	age_in_years INTEGER, 
	gender VARCHAR(17), 
	id TEXT NOT NULL, 
	name TEXT, 
	description TEXT, 
	image TEXT, 
	"Container_id" INTEGER, 
	current_address_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY("Container_id") REFERENCES "Container" (id), 
	FOREIGN KEY(current_address_id) REFERENCES "Address" (id)
);
CREATE TABLE "Organization" (
	mission_statement TEXT, 
	founding_date TEXT, 
	founding_location TEXT, 
	id TEXT NOT NULL, 
	name TEXT, 


### Create an empty database

Ensuring we start with a fresh database, we initialize it with CREATE TABLE STATEMENTS

In [5]:
!test -f tmp/tmp.db && rm tmp/tmp.db

In [6]:
!cat tmp/personinfo.ddl.sql | sqlite3 tmp/tmp.db 

In [7]:
!sqlite3 tmp/tmp.db ".schema Person" ".exit"

CREATE TABLE IF NOT EXISTS "Person" (
	primary_email TEXT, 
	birth_date TEXT, 
	age_in_years INTEGER, 
	gender VARCHAR(17), 
	id TEXT NOT NULL, 
	name TEXT, 
	description TEXT, 
	image TEXT, 
	"Container_id" INTEGER, 
	current_address_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY("Container_id") REFERENCES "Container" (id), 
	FOREIGN KEY(current_address_id) REFERENCES "Address" (id)
);


## Generate SQLA Models

We use the [gen-sqla](https://linkml.io/linkml/generators/sqlalchemy.html) command to create a SQLA Object Model

In [8]:
!gen-sqla ../examples/PersonSchema/personinfo.yaml > tmp/personinfo_sqla.py

In [9]:
!cat tmp/personinfo_sqla.py


from sqlalchemy import Column, Index, Table, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.sql.sqltypes import *
from sqlalchemy.orm import declarative_base
from sqlalchemy.ext.associationproxy import association_proxy

Base = declarative_base()
metadata = Base.metadata


class NamedThing(Base):
    """
    A generic grouping for any identifiable entity
    """
    __tablename__ = 'NamedThing'

    id = Column(Text(), primary_key=True, nullable=False )
    name = Column(Text())
    description = Column(Text())
    image = Column(Text())
    

    def __repr__(self):
        return f"NamedThing(id={self.id},name={self.name},description={self.description},image={self.image},)"



    


class HasAliases(Base):
    """
    A mixin applied to any class that can have aliases/alternateNames
    """
    __tablename__ = 'HasAliases'

    id = Column(Integer(), primary_key=True, autoincrement=True , nullable=False )
    
    
    aliases_rel = relationship( "HasAliasesAlia

## Populate the SQLA Model Objects

We will import our object model (remember in production code we would not put this in `tmp`).

We will also use the Jupyter builtin help to introspect a class - although the builtin SQLA methods make this
a bit verbose and messy

In [10]:
from tmp.personinfo_sqla import Person
help(Person)

Help on class Person in module tmp.personinfo_sqla:

class Person(NamedThing)
 |  Person(**kwargs)
 |  
 |  A person (alive, dead, undead, or fictional).
 |  
 |  Method resolution order:
 |      Person
 |      NamedThing
 |      sqlalchemy.orm.decl_api.Base
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, **kwargs)
 |      A simple constructor that allows initialization from kwargs.
 |      
 |      Sets attributes on the constructed instance using the names and
 |      values in ``kwargs``.
 |      
 |      Only keys that are present as
 |      attributes of the instance's class are allowed. These could be,
 |      for example, any mapped columns or relationships.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  Container_id
 |  
 |  age_in_years
 |  
 |  aliases_rel
 |  
 |  birth_date
 |  
 |  current_address
 |  
 |  current_ad

### Create a SQLA session

This is a bit involved, but this is all standard for SQLA

In [11]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [12]:
engine = create_engine("sqlite:///tmp/tmp.db")

In [13]:
session_class = sessionmaker(bind=engine)
session = session_class()

### Create an object and add it

We will populate the database with a single person, then do a sanity check SQL query

In [14]:
p = Person(id="P1", name="Rosario", age_in_years=33)

In [15]:
session.add(p)

In [16]:
session.commit()

In [17]:
!sqlite3 tmp/tmp.db "SELECT * FROM Person;" ".exit"

||33||P1|Rosario||||


## Foreign Keys

Now we demonstrate use of foreign keys behind the scenes.

In the LinkML model, FamilialRelationship is *inlined*, so each Person can have zero to many embedded relationships
to other people.

We will append a relationship to another person (P2 - note this person is not yet in the database,
but it's OK as SQLite3 doesn't enforce referential integrity)

In [18]:
from tmp.personinfo_sqla import FamilialRelationship

In [19]:
p.has_familial_relationships.append(FamilialRelationship(related_to="P2", type="SIBLING_OF"))

In [20]:
session.commit()

In [21]:
!sqlite3 tmp/tmp.db "SELECT * FROM FamilialRelationship;" ".exit"

1|||P2|SIBLING_OF|P1


### Generating synthetic data

Next we will write some simple code for making synthetic people

In [22]:
%pip install names

  Using cached names-0.3.0-py3-none-any.whl


Note: you may need to restart the kernel to use updated packages.


In [23]:
import names
import random

last_id = 2
def random_person() -> Person:
    """
    Create a person with a random name and random set of relationships.
    
    Note that this is just for demo purposes. If we wanted to be more advanced we
    could make use of SIBLING_OF to make a more realistic distribution of last name
    matches, and we could ensure that SIBLING_OF is symmetric...
    """
    global last_id
    last_id += 1
    pid = f"P{last_id}"
    person = Person(id=pid, name=names.get_full_name(), age_in_years=random.randint(30,40))
    for n in range(0, random.randint(0, 3)):
        rel = FamilialRelationship(related_to=f"P{last_id+n}", type="SIBLING_OF")
        person.has_familial_relationships.append(rel)
    return person

In [24]:
persons = [random_person() for x in range(1,100)]

### Container Object

The PersonInfo schema has a class Container whose sole function is to contain all top level objects.

We will populate our container with our 100 artificial people.

In [25]:
from tmp.personinfo_sqla import Container

In [26]:
container = Container(persons=persons)

In [27]:
session.add(container)

In [28]:
session.commit()

In [29]:
!sqlite3 tmp/tmp.db "SELECT * FROM FamilialRelationship LIMIT 10;" ".exit"

1|||P2|SIBLING_OF|P1
2|||P3|SIBLING_OF|P3
3|||P4|SIBLING_OF|P3
4|||P5|SIBLING_OF|P3
5|||P4|SIBLING_OF|P4
6|||P5|SIBLING_OF|P4
7|||P6|SIBLING_OF|P4
8|||P5|SIBLING_OF|P5
9|||P6|SIBLING_OF|P5
10|||P7|SIBLING_OF|P5


## Use of SQLA ORM to query

Next we will demonstrate use of the SQLA Object-Relational Mapper to query for the first 10 objects in the database.

Note that the ORM takes care of writing queries involving joins for us, so inlined objects are retrieved naturally

In [30]:
for p in session.query(Person)[0:10]:
    print(p.name)
    for frel in p.has_familial_relationships:
        print(f"  {frel.type} {frel.related_to}")

Rosario
  SIBLING_OF P2
Deandre Weis
  SIBLING_OF P3
  SIBLING_OF P4
  SIBLING_OF P5
Leona Greer
  SIBLING_OF P4
  SIBLING_OF P5
  SIBLING_OF P6
Ella Holt
  SIBLING_OF P5
  SIBLING_OF P6
  SIBLING_OF P7
Catherine Miller
  SIBLING_OF P6
  SIBLING_OF P7
Frances Permenter
Arthur Flynn
  SIBLING_OF P8
  SIBLING_OF P9
Richard Nein
  SIBLING_OF P9
  SIBLING_OF P10
Ruth Razo
Carl Jones
  SIBLING_OF P11
  SIBLING_OF P12
  SIBLING_OF P13


## Bridging SQLA Models with LinkML Dataclasses or Pydantic

If you are already familiar with LinkML use with Python, you will likely be familiar with:

- [gen-python](https://linkml.io/linkml/generators/python.html)
- [gen-pydantic](https://linkml.io/linkml/generators/pydantic.html)

Which create dataclasses and pydantic object models respectively.

Unfortunately there is no one single standard for object models in Python. Dataclasses and Pydantic have a lot of advantages over SQLA models, but they don't work as well in conjunction with the ORM.

To accommodate this, the LinkML python toolchain has bridging code that will map objects from one system to another.

In [31]:
!gen-pydantic ../examples/PersonSchema/personinfo.yaml > tmp/personinfo.py

In [32]:
import tmp.personinfo as personinfo

In [33]:
personinfo.FamilialRelationship(related_to="P2", type=personinfo.FamilialRelationshipType("SIBLING_OF")).model_dump()

{'started_at_time': None,
 'ended_at_time': None,
 'related_to': 'P2',
 'type': 'SIBLING_OF'}

In [34]:
def random_person_pydantic() -> personinfo.Person:
    """
    Create a person with a random name and random set of relationships.
    
    Note that this is just for demo purposes. If we wanted to be more advanced we
    could make use of SIBLING_OF to make a more realistic distribution of last name
    matches, and we could ensure that SIBLING_OF is symmetric...
    """
    global last_id
    last_id += 1
    pid = f"P{last_id}"
    person = personinfo.Person(id=pid, name=names.get_full_name(), age_in_years=random.randint(30,40))
    for n in range(0, random.randint(0, 3)):
        rel = personinfo.FamilialRelationship(related_to=f"P{last_id+n}", type="SIBLING_OF")
        relationships = person.has_familial_relationships
        if relationships:
            relationships.append(rel)
        else:
            person.has_familial_relationships = [rel]
            
    return person

In [35]:
random_person_pydantic()

Person(id='P102', name='Tina Burton', description=None, image=None, aliases=None, primary_email=None, birth_date=None, age_in_years=36, gender=None, current_address=None, has_employment_history=None, has_familial_relationships=[FamilialRelationship(started_at_time=None, ended_at_time=None, related_to='P102', type='SIBLING_OF')], has_medical_history=None)

In [36]:
container = personinfo.Container(persons=[random_person_pydantic() for x in range(1,100)])

In [37]:
from linkml.utils.sqlutils import SQLStore

In [38]:
store = SQLStore(use_memory=True)

In [39]:
store.module = personinfo

In [40]:
container_sqla = store.to_sqla(container)

In [41]:
type(container_sqla)

tmp.personinfo.Container